# Initial Setup

## Install Weaviate Python Client v4
> This notebook was created with Weaviate `1.25` and the Weaviate Client `4.6`

In [ ]:
!pip install weaviate-client==4.6.1 --force-reinstall -U -q

## Prepare API keys

### Make sure Weaviate IP Key gets printed!

In [ ]:
print(f'Weaviate IP: {weaviate_ip}')

In [ ]:
from boto3 import Session

session = Session()
credentials = session.get_credentials()
current_credentials = credentials.get_frozen_credentials()

AWS_ACCESS_KEY = current_credentials.access_key
AWS_SECRET_KEY = current_credentials.secret_key
AWS_SECRET_TOKEN = current_credentials.token
print(f"AWS_ACCESS_KEY:\t{AWS_ACCESS_KEY}")
print(f"AWS_SECRET_KEY:\t{AWS_SECRET_KEY}")
print(f"AWS_SECRET_TOKEN:\t{AWS_SECRET_TOKEN}")
print(f"WEAVIATE_IP:\t{WEAVIATE_IP}")

## Deploy Weaviate

Weaviate offers 3 deployment options:
* Self-hosted - with Docker Compose, Kubernetes
* Cloud deployment - [Weaviate Cloud](https://console.weaviate.cloud/)
* Embedded

# Time to Build

## Connect to Weaviate

In [ ]:
import weaviate

client = weaviate.connect_to_custom(
    http_host=WEAVIATE_IP, http_port="8080",  http_secure=False,
    grpc_host=WEAVIATE_IP, grpc_port="50051", grpc_secure=False,

    headers={
        "X-AWS-Access-Key": AWS_ACCESS_KEY,
        "X-AWS-Secret-Key": AWS_SECRET_KEY,
        "X-AWS-Session-Token": AWS_SECRET_TOKEN,
    }
)

## Create a collection
[Weaviate Docs - collection creation and configuration](https://weaviate.io/developers/weaviate/configuration/schema-configuration)

In [ ]:
from weaviate.classes.config import Configure

if client.collections.exists("Jeopardy"):
    client.collections.delete("Jeopardy")

# Create a collection here
client.collections.create(
    name="Jeopardy",

    # Create a collection - with AWS vectorizer and Titan model("amazon.titan-embed-text-v1")
    vectorizer_config=Configure.Vectorizer.text2vec_aws(
        model="amazon.titan-embed-text-v1",
        region="us-west-2",
        # region="us-east-1",
    ),
)

## Import data

### Sample Data

In [ ]:
import json

with open("./jeopardy-100.json") as file:
    data_100 = json.load(file)

print(json.dumps(data_100[0], indent=2))

### Insert Many
[Weaviate Docs - insert many](https://weaviate.io/developers/weaviate/manage-data/import)

In [ ]:
# Insert data
jeopardy = client.collections.get("Jeopardy")
jeopardy.data.insert_many(data_100)

## Check the object count
[Weaviate Docs - Aggregate](https://weaviate.io/developers/weaviate/search/aggregate#retrieve-the-count-meta-property)

In [ ]:
questions = client.collections.get("Jeopardy")
questions.aggregate.over_all()

### Data preview

In [ ]:
# Show data preview
jeopardy = client.collections.get("Jeopardy")
response = jeopardy.query.fetch_objects(limit=4)

for item in response.objects:
    print(item.uuid, item.properties)

In [ ]:
# Show data preview - with vectors
jeopardy = client.collections.get("Jeopardy")
response = jeopardy.query.fetch_objects(
    limit=4,
    include_vector=True
)

for item in response.objects:
    print(item.properties)
    print(item.vector, '\n')

## Close the client when done

In [ ]:
client.close()